In [153]:
import csv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd
import os.path

data_sufix = 'data/'

In [154]:
# configure Session class with desired options
Session = sessionmaker()
engine = create_engine('sqlite:///twitter.db')

In [155]:
engine.table_names()

[]

In [156]:
hashtags = ['h1b', 'usvisa', 'illegalImmigration', 'immigration',
'immigrantcaravan', 'caravaninvasion', 'buildthewall', 'illegalaliens']
hashtagsDict = {'name': hashtags}
unbalancedHashtags = ['birthrightcitizenship', 'immigrationreform']

In [157]:
hashtagData = pd.DataFrame.from_dict(hashtagsDict)
hashtagData.to_sql('hashtag', engine, if_exists='replace', index=True, index_label='id')

In [158]:
valid_dates = ['2018-10-27','2018-10-28','2018-10-29','2018-10-30']

In [159]:
for hashtag in hashtags:
    
    hashtag_id = pd.read_sql_query('SELECT id FROM hashtag WHERE hashtag.name="'+hashtag+'"', engine).values[0][0]

    tweetData = pd.read_csv(data_sufix+hashtag+'-tweet.csv', 
                            usecols=['tweet_id','tweet_text','tweet_created_at','tweet_retweet_count',
                                     'tweet_favorite_count','user_id'])
    tweetData['hashtag_id'] = hashtag_id
    
    retweetData = pd.read_csv(data_sufix+hashtag+'-retweet.csv', 
                              usecols=['retweet_id','source_tweet_id','retweet_text','retweet_created_at','user_id'])
    
    tweetData = tweetData.loc[tweetData['tweet_created_at'].str.contains('|'.join(valid_dates))]
    retweetData = retweetData.loc[retweetData['retweet_created_at'].str.contains('|'.join(valid_dates))]
    
    tweetData.to_sql('tweet', engine, if_exists='append')
    retweetData.to_sql('retweet', engine, if_exists='append')

In [160]:
userData = pd.read_csv(data_sufix+'user.csv', 
                        usecols=['user_id','user_name','user_location','user_friends_count','followers_count'])

In [161]:
valid_tweeting_user_ids = pd.read_sql_query('SELECT DISTINCT user_id FROM tweet', engine)
valid_retweeting_user_ids = pd.read_sql_query('SELECT DISTINCT user_id FROM retweet', engine)

In [162]:
valid_user_ids = valid_tweeting_user_ids['user_id'].tolist() + valid_retweeting_user_ids['user_id'].tolist()

In [163]:
len(valid_user_ids)

44128

In [164]:
userData = userData.loc[userData['user_id'].isin(valid_user_ids)]

In [165]:
userData.to_sql('user', engine, if_exists='append')

In [166]:
#HASHTAGS

allTweets = pd.read_sql_query('SELECT tweet_id as id, tweet_text as text FROM tweet', engine)
print("tweets " + str(allTweets.shape))
allRetweets =  pd.read_sql_query('SELECT retweet_id as id, retweet_text as text FROM retweet', engine)
print("retweets " + str(allRetweets.shape))
textData = allTweets.append([allRetweets])
textData.drop_duplicates(inplace=True)
textData[:5]

tweets (17596, 2)
retweets (79436, 2)


,id,text
0,1056095192930025472,b'@Earlybird0073 @immivoice I guess #h1b #h4ea...
1,1056125697482256384,b'@AmerMedicalAssn @USCIS #kansas #traitor @Ke...
2,1056125759851556864,b'@AmerMedicalAssn @USCIS @KevinYoder sneaked ...
3,1056125800196595712,b'@AmerMedicalAssn @USCIS #HR392 has been snea...
4,1056125844966592512,b'@AmerMedicalAssn @USCIS 150 year #Backlog of...


In [ ]:
import re
hashes=''
regex = r"#([a-zA-Z\d]*)"
for index, row in textData.iterrows():
    match = re.findall(regex, row["text"])
    hashes = list(set(hashes)) + [x.lower() for x in match]
# print(hashes)
len(hashes)

In [ ]:
allHashes = pd.DataFrame({'hashtag':hashes})
allHashes.to_sql('hashes', engine, if_exists='append')

In [ ]:
allDBHashes = pd.read_sql_query('SELECT * FROM hashes', engine)
print(allDBHashes.shape)
allDBHashes[:5]

In [ ]:
import re
hashes=''
regex = r"#([a-zA-Z\d]*)"
tweetToHashes = pd.DataFrame(columns=['tweet_id', 'hash_id'])
for index, row in textData.iterrows():
    match = re.findall(regex, row["text"])
    hashes = [x.lower() for x in match]
    tweetID = row["id"]
    for hsh in hashes:
        hashID = allDBHashes[allDBHashes['hashtag'] == hsh].iloc[0]["index"]
        tweetToHashes.loc[len(tweetToHashes)]=[tweetID, int(hashID)]
tweetToHashes

In [ ]:
tweetToHashes.to_sql('tweetToHashes', engine, if_exists='append')

In [ ]:
# associate it with our custom Session class
Session.configure(bind=engine)